move onto torch/models
get protein model running
come back to chem/templates, remake with protein variants. code will hopefully be simpler


Clean up docs
make tutorial for advanced templates (blocks)
get collab working
get google search working

torch core
models (lstm, vae, transformer)
    work in protein stuff here

score functions

training loop

poicy gradients

q-network

diff-loss

exploration strategies

combichem

pharmacophore 


pages
overview

generrative screening primerr